In [1]:
import os
os.chdir("../../..")

# Quickstart

In this tutorial, for computation time issues (e.g. training a neural network is time consumming), we will use a small time series forecasting task from the **GluonTS** package, called *m1_monthly*.


More applications are detailed here: [Examples](../Examples/index.rst)

### Loading the dataset

In [2]:

from dragon.experiments.monash_archive.dataset import gluonts_dataset
from dragon.experiments.monash_archive.datasets_configs import m1_monthly_config

train_ds, test_ds, config = gluonts_dataset(m1_monthly_config)
config['SaveDir'] = config['PathName'] + "_"

2023-10-13 11:54:34,035 | INFO | To use MPILoss object you need to install mpi4py and an MPI distribution
    You can use: pip install zellij[MPI]
2023-10-13 11:54:34,190 | INFO | Started loading m1_monthly


### Defining the Loss Function

The `loss function` measures how well our Deep Neural Network performed on the considered task. This function should handle the DNN training procedure as well as the metric computation on the validation set. The user is required to define a model, a training and a testing procedure.

Then, we use a wrapping function called `zellij.core.Loss` from **Zellij**. By setting the argument `MPI` to **True**, one can use the distributed version of the `Loss` object.


##### DNN definition

The class `dragon.experiments.monash_archive.training.GluontsNet`, designed specially for the **GluonTS** forecasting datasets handles the DNN creation, its training and testing procedure.

In [3]:
from dragon.experiments.monash_archive.training import GluontsNet
m1_monthly_config['NumEpochs'] = 20
model = GluontsNet(train_ds, test_ds, m1_monthly_config)

##### Loss function definition

In [4]:

import numpy as np
from zellij.core import Loss    

loss = Loss(MPI=False, verbose=False, save=True)(model.get_nn_forecast)

### Search Space definition

To define a searchspace one need to define variables `var`, which would be optimized.

The DNN are modelized by **AdjMatrix** (`dragon.search_space.dags.AdjMatrixVariable`). They are parametrized by a set of candidate operations. Each candidate operations are modelized by an **ArrayVar** (`zellij.core.ArrayVar`) containing the operation name and the associated *hyperparameters*. They can be of type:

* **Floats**: `zellij.core.FloatVar`, e.g: learning rate, dropout rate, etc.
* **Integers**: `zellij.core.IntVar`, e.g: output dimension, kernel size, etc.
* **Categorical**: `zellij.core.CatVar`, e.g: activation function, pooling type, etc.

Typical candidate operations variables are already defined within the package: `dragon.search_space.variables`. They are based on `nn.Module` defined in the `dragon.search_space.bricks` repository.

In [5]:
from zellij.core.variables import CatVar, ArrayVar, DynamicBlock
from zellij.utils.neighborhoods import ArrayInterval, DynamicBlockInterval

from dragon.search_algorithm.neighborhoods import LayersInterval, AdjMatrixInterval
from dragon.search_space.dags import AdjMatrixVariable
from dragon.search_space.variables import unitary_var, mlp_var, activation_var, create_int_var

# We define the candidate operations for each nodes in the graph. Here we only consider multi-layers perceptron and identity operations.
def operations_var(label, shape, size):
    return DynamicBlock(
        label,
        CatVar(
            label + "Candidates",
            [
                unitary_var(label + " Unitary"),
                mlp_var(label + " MLP"),
            ],
            neighbor=LayersInterval([2, 1]),
        ),
        size,
        neighbor=DynamicBlockInterval(neighborhood=2),
    )

# We define the serach space, a graph handling one-dimensional data, and the final activation function before the prediction.
def NN_monash_var(label="Neural Network", shape=1000, size=10):
    NeuralNetwork = ArrayVar(
        AdjMatrixVariable(
            "Cell",
            operations_var("Feed Cell", shape, size),
            neighbor=AdjMatrixInterval()
        ),
        activation_var("NN Activation"),
        create_int_var("Seed", None, 0, 10000),
        label=label,
        neighbor=ArrayInterval(),
    )
    return NeuralNetwork

sp = NN_monash_var(shape=m1_monthly_config["Lag"], size=3)

Once your search space is defined, you can draw random points:

In [6]:
p1,p2 = sp.random(), sp.random()
print("First random point: ", p1)
print("Second random point: ", p2)


First random point:  [NODES: [['Input'], ['concat', 'MLP', 386, 'swish']] | MATRIX:[[0, 1], [0, 0]], 'sigmoid', 2618]
Second random point:  [NODES: [['Input'], ['add', 'Identity'], ['concat', 'Identity']] | MATRIX:[[0, 1, 1], [0, 0, 1], [0, 0, 0]], 'leaky_relu', 3127]


Now we can use the loss function on the search space:

In [7]:
scores = loss([p1, p2])




Global seed set to 2618
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [9]:
model.config['NumEpochs']

100